In [1]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.6 MB/s eta 0:00:00


In [2]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import random
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
import multiprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import json
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from gensim.models import Word2Vec
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import accuracy_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ProjectsAI_CodeInjection/Dataset

data_source = pd.read_csv('data_source.csv')
data_target  = pd.read_csv('data_target.csv')

data_target = data_target.sample(frac=1, random_state=42)

X_source = data_source['text'].str.replace('/',' ')
y_source = data_source['label']
X_target  = data_target ['text'].str.replace('/','')
y_target  = data_target ['label']
rlist =['194 - Fake the Source of Data',
        '66 - SQL Injection',
        '34 - HTTP Response Splitting',
        '126 - Path Traversal',
        '000 - Normal',
        '272 - Protocol Manipulation',
        '310 - Scanning for Vulnerable Software',
        '242 - Code Injection',
        '153 - Input Data Manipulation']
mapping = {l: i+1 for i, l in enumerate(rlist)}
y_source = [mapping[s] for s in y_source]
y_target  = [mapping[r] for r in y_target ]
y_source = np.array(y_source)
y_target  = np.array(y_target )

Mounted at /content/drive
/content/drive/MyDrive/ProjectsAI_CodeInjection/Dataset


In [4]:
len(X_source), len(X_target)

(31552, 33343)

# BERT Feature Extraction

In [5]:
model_name = 'jackaduma/SecBERT'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_1 = AutoModel.from_pretrained(model_name)
model_2 = AutoModel.from_pretrained(model_name)

for name, param in model_1.named_parameters():
  # if "5" in name or "4" in name or "3" in name:
    param.requires_grad = False
for name, param in model_2.named_parameters():
  # if "5" in name or "4" in name or "3" in name:
    param.requires_grad = False

X_source = list(X_source)
X_target = list(X_target)
input_texts_source = X_source
input_text_target = X_target

X_train = []
X_test = []

for i , text in enumerate(input_texts_source):

    tokens = tokenizer.tokenize(text)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_tensor = torch.tensor([token_ids])

    if (input_tensor.shape[1] > 512):
      input_tensor = input_tensor[:, :512]
    else:
      input_tensor = torch.cat((input_tensor, torch.zeros((1, 512 - input_tensor.shape[1]))), 1)

    X_train.append(input_tensor.squeeze())
    if i%1000==0 :
        print(i/len(X_source))

for j , text in enumerate(input_text_target):

    tokens = tokenizer.tokenize(text)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_tensor = torch.tensor([token_ids])

    if (input_tensor.shape[1] > 512):
      input_tensor = input_tensor[:, :512]
    else:
      input_tensor = torch.cat((input_tensor, torch.zeros((1, 512 - input_tensor.shape[1]))), 1)

    X_test.append(input_tensor.squeeze())
    if j%1000==0 :
        print(j/len(X_target))

X_test = torch.stack(X_test)
X_train = torch.stack(X_train)

0.0
0.03169371196754564
0.06338742393509128
0.09508113590263692
0.12677484787018256
0.1584685598377282
0.19016227180527384
0.22185598377281948
0.2535496957403651
0.28524340770791073
0.3169371196754564
0.348630831643002
0.3803245436105477
0.4120182555780933
0.44371196754563896
0.4754056795131846
0.5070993914807302
0.5387931034482759
0.5704868154158215
0.6021805273833671
0.6338742393509128
0.6655679513184585
0.697261663286004
0.7289553752535497
0.7606490872210954
0.7923427991886409
0.8240365111561866
0.8557302231237323
0.8874239350912779
0.9191176470588235
0.9508113590263692
0.9825050709939148
0.0
0.029991302522268542
0.059982605044537084
0.08997390756680562
0.11996521008907417
0.1499565126113427
0.17994781513361124
0.2099391176558798
0.23993042017814833
0.26992172270041687
0.2999130252226854
0.329904327744954
0.3598956302672225
0.38988693278949105
0.4198782353117596
0.4498695378340281
0.47986084035629667
0.5098521428785652
0.5398434454008337
0.5698347479231023
0.5998260504453709
0.62981

In [6]:
X_test.shape, X_train.shape

(torch.Size([33343, 512]), torch.Size([31552, 512]))

# Convert Dữ liệu chưa biết có nhãn là 0, đã biết nhãn là 1

In [7]:
y_target = np.where(np.logical_or(y_target == 1, y_target == 3), 0, 1)

# GPU

In [8]:
device = torch.device("cpu")

# Set giá trị đầu vào

In [9]:
input_layer = X_train.shape[1]
input_size = input_layer
hidden_size =64

# Hàm Discriminator

In [10]:
class Discriminator(nn.Module):
    def __init__(self, bert):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(768, 1024),
            # nn.ReLU(),
            # nn.Dropout(0.3),
            # nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )
        self.bert = bert

    def forward(self, x):
        x = torch.round(x).type(torch.IntTensor).to(device=device)
        lm_outputs = self.bert(x)
        last_hidden_state = lm_outputs.last_hidden_state
        first_token_features = last_hidden_state[:, 0, :]
        output = self.model(first_token_features)
        # output = output.view(x.size(0), 512)
        return output

discriminator = Discriminator(model_1).to(device=device)


# Hàm Generator

In [11]:
class Generator(nn.Module):
    def __init__(self, bert):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(768, 256), # max_length: 512
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.Tanh(),
        )
        self.bert = bert

    def forward(self, x):
        x = torch.round(x).type(torch.IntTensor).to(device=device) # Evil trick;
        lm_outputs = self.bert(x)
        last_hidden_state = lm_outputs.last_hidden_state
        first_token_features = last_hidden_state[:, 0, :]
        output = self.model(first_token_features)
        output = output.view(x.size(0), 512)
        return output

generator = Generator(model_2).to(device=device)

# Khởi tạo các giá trị laerning rate, epochs, batch_size, hàm loss, optimization

In [12]:
lr = 0.0001
num_epochs = 1
batch_size= 2
loss_function = nn.BCELoss()
optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr)
optimizer_generator = torch.optim.Adam(generator.parameters(), lr=lr)

# Dùng để chuyển đổi dạng đầu vào

In [13]:
X_source = X_train.to(device=device, dtype=torch.float32)
X_target = X_test.to(device=device, dtype=torch.float32)

# Trainning

In [14]:
import torch
torch.cuda.empty_cache()

In [ ]:
for epoch in range(num_epochs):
    idx = np.random.randint(0, X_source.shape[0], batch_size)
    real_samples  = X_source[idx]
    real_samples_labels = torch.ones((batch_size,1)).to(device=device)
    latent_space_samples = torch.randint(0, 52000, (batch_size,512)).to(device=device)

    generated_samples = generator(latent_space_samples)
    generated_samples_labels = torch.zeros((batch_size, 1)).to(
            device=device
        )
    all_samples = torch.cat((real_samples, generated_samples))
    all_samples_labels = torch.cat(
            (real_samples_labels, generated_samples_labels)
        )

        # Training the discriminator
    discriminator.zero_grad()

    output_discriminator = discriminator(all_samples)
    all_samples_labels = all_samples_labels.view(-1, 1)
    loss_discriminator = loss_function(
            output_discriminator, all_samples_labels
        )
    loss_discriminator.backward()
    optimizer_discriminator.step()

        # Data for training the generator
    latent_space_samples = torch.rand((batch_size,512)).to(device=device)

        # Training the generator
    generator.zero_grad()
    generated_samples = generator(latent_space_samples)
    output_discriminator_generated = discriminator(generated_samples)
    loss_generator = loss_function(
            output_discriminator_generated, real_samples_labels
        )
    loss_generator.backward()
    optimizer_generator.step()
    if(epoch%100==0):
        predicted_labels = discriminator(X_target)
        new_tensor = torch.where(predicted_labels >= 0.5, torch.tensor(1), torch.tensor(0))
        new_label = new_tensor.numpy()

        # print(f"accuracy: {accuracy_score(y_target, new_label)}")
        print(f"Epoch: {epoch/100} Loss D.: {loss_discriminator}")
        print(f"Epoch: {epoch/100} Loss G.: {loss_generator}")

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


# chuyển tensor output về 0 và 1

In [ ]:
predicted_labels = discriminator(X_target)
new_tensor = torch.where(predicted_labels >= 0.5, torch.tensor(1), torch.tensor(0))

In [ ]:
new_label = new_tensor.numpy()

# Kết quả phân loại trên tập test

In [ ]:
accuracy = accuracy_score(y_target[0:10], new_label)
accuracy

In [ ]:
print('accuracy =', accuracy_score(y_target[0:10], new_label))
print(classification_report(y_target[0:10],new_label))
print('Confusion Matrix: \n',confusion_matrix(y_target[0:10], new_label))